## 設定POST 的Form Data

In [2]:
import requests

payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'103/04/29',
 'awardAnnounceEndDate':'103/04/29', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

## 送出Post 取得資料

In [3]:
user_post = requests.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8')  #為何要用encode ?? 用了encode ..好像沒差別
#print response_text

## 如何延續Session

In [4]:
#為何需要延續Session ?? 什麼情況之下 ??

rs = requests.session() 
rs_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
rs_get = rs.get("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2") 
response = rs_get.text.encode('utf8')
#print response


## 取得資料筆數

In [5]:
from bs4 import BeautifulSoup

user_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload)
response_text = user_post.text.encode('utf8') 
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select('.T11b')[0]
#print int(rec_number_element.text) 
rec_number = int(rec_number_element.text) 
#print rec_number

D:\Users\alone\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


## 將資料筆數轉換成頁數

In [6]:
#-----------------------------------------------------
from math import ceil 
#int(ceil(float(rec_number) / 100))
page_number = int(ceil(float(rec_number) / 100)) 
#print page_number


## 字串格式化 - 使用format

In [7]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex={0}" 
for page in range(1, 10 + 1): 
    print page_format.format(page)

#--------------------------------------------
# 使用%
#page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
#for page in range(1, 5 + 1): 
#    print page_format%(page)
#--------------------------------------------

http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=1
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=3
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=4
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=5
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=6
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=7
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=sea

## 取得每頁標案清單的內容

In [8]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8')
    #print bid_response

## 先拿一頁做試驗

In [9]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
page = 1 
bid_list = rs.get(page_format%(page)) 
bid_response = bid_list.text.encode('utf8') 
bid_soup = BeautifulSoup(bid_response) 
#print bid_response
bid_table = bid_soup.select('#print_area')[0]
#print bid_table

## 去掉標頭 去掉標頭跟頁次 索引範例

In [10]:
bid_rows = bid_table.select('tr')[1:] 
#print bid_rows
bid_rows = bid_table.select('tr')[1:-1] 
#print bid_rows
ary = [1,2,3,4,5,6] 
print ary[1:-1] 
#[2,3,4,5]

[2, 3, 4, 5]


## 抓出每一列所有的連結  (重點!!!!)

In [11]:
#print bid_table
bid_rows = bid_table.select('tr')[1:-1] 
for bid_row in bid_rows: 
    links = [tag['href']for tag in bid_row.select('a')][0]  # 為何是用tag ??????????? ...取第一個連結
    #print links

## 取得實際連結

In [29]:
# Test !!!!!

#print bid_rows
#for bid_row in bid_rows: 
    #link = [tag['align'] for tag in bid_row.findAll('td',{'align': True})][0]  #同時出現td align 才顯示
    #link = [tag['align'] for tag in bid_row.findAll('td',{'align': True})][1]
    #print link
#--------------------------------


#print bid_rows
for bid_row in bid_rows: 
    link = [tag['href'] for tag in bid_row.findAll('a',{'href': True})][0]   # 為何是用True?????
    #print link
    link_href = "http://web.pcc.gov.tw/tps" + link[2:]  # [2:] 去除前面兩個字元
    #print link_href

## 使用urljoin

In [31]:
import urlparse
for bid_row in bid_rows: 
    link = [tag['href'] 
      for tag in bid_row.findAll('a',{'href': True})][0] 
    link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link)
    
    #print link_href

## 複習一下如何使用Python做檔案寫入

In [33]:
f = open("test.txt", 'w') 
f.write("Hello World\n") #每次執行 都會以新的檔案 寫入..即只能ㄧ次寫入。
f.close() 

## 將每頁標案清單的連結存入檔案

In [62]:
bid_file = open("bid_list.txt",'w')

#page_format ????
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 

for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page))  
    #print bid_list  #=>reponse 200
    bid_response = bid_list.text.encode('utf8')
    #print bid_response
    bid_soup = BeautifulSoup(bid_response) 
    #print bid_soup
    
    # Error
        #bid_table = bid_response.select("#print_area") [0] # str object has no attribute 'select'
        #print bid_table
        
    bid_table = bid_soup.select("#print_area") [0]
    #print bid_table
    bid_rows = bid_table.select('tr')[1:-1]
    #print bid_rows
    
    for bid_row in bid_rows:         
        link = [tag['href'] for tag in bid_row.select('a')][0] 
        link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link) 
        bid_file.write(link_href + "\n") 
bid_file.close()  # 檔名為 bid_list.txt

## 0. 引入該用的package

In [63]:
# -*- coding: utf-8 -*- 
import requests
import urlparse 
from bs4 import BeautifulSoup 
from math import ceil

## 1. 設定查詢條件

In [64]:
payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'103/04/29',
 'awardAnnounceEndDate':'103/04/29', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

rs = requests.session() 
user_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8')

## 2. 計算抓取頁數

In [65]:

soup = BeautifulSoup(response_text) 
rec_number_element = soup.select(".T11b" )[0] 
rec_number = int(rec_number_element.text)
page_number = int(ceil(float(rec_number) / 100))

## 3. 依每頁抓取所有標案連結 & 4. 存入每頁標案連結

In [67]:
bid_file = open("bid_list.txt", 'w') 
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8') 
    bid_soup = BeautifulSoup(bid_response)
    bid_table = bid_soup.select("#print_area")[0] 
    bid_rows = bid_table.select('tr')[1:-1] 
    for bid_row in bid_rows: 
        link = [tag['href'] for tag in bid_row.select('a')][0] 
        link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link) 
        bid_file.write(link_href + "\n") 
bid_file.close() 

## 計算總共存了多少個連結

In [68]:
line_num = 0 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    line_num = line_num + 1 
print line_num 
f.close() 

264


## 判斷抓下來的資料筆數 是否與網頁上呈現的相同

In [69]:
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select(".T11b" )[0] 
rec_number = int(rec_number_element.text)

line_num = 0 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    line_num = line_num + 1 
print line_num 
f.close() 

#使用if 判斷
if rec_number == line_num: 
    print "record number on web page is equal to number of lines in file"

264
record number on web page is equal to number of lines in file


## 使用requests.get 抓取標案細節 

In [70]:
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328") 
response = request_get.text.encode('utf8') 
#print response

## 取得id為printArea 的區塊

In [72]:
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328") 
response = request_get.text.encode('utf8') 
soup = BeautifulSoup(response)
printarea = soup.select('#printArea') [0]
#print printarea

## 網址下載完後之處理 ~ 將標案資料寫進檔案中
## Error  資料沒寫入到檔案內...解決方法在下面..

In [87]:
bid_detail = open("bid_detail.txt", 'w') 
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328") 
response = request_get.text.encode('utf8')
#print response
soup = BeautifulSoup(response)
printarea = soup.select('#printArea')[0]
#print type(printarea) # class  bs4.element.Tag

bid_detail.write(printarea) ### expected a string or other character buffer object
bid_detail.close()

TypeError: expected a string or other character buffer object

## 使用prettify 將soup 抓到的tag 轉換成str

In [90]:
bid_detail = open("bid_detail.txt", 'w') 
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328") 
response = request_get.text.encode('utf8') 
soup = BeautifulSoup(response)
printarea = soup.select('#printArea') [0] 
bid_detail.write(printarea.prettify("utf-8")) 
bid_detail.close()

## 怎麼找出案號是否會重複?

In [92]:
case_dic = {} 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    rec = line.strip() 
    caseno = rec.split('tenderCaseNo=')[1] 
    if caseno not in case_dic: 
        case_dic[caseno] = 1 
    else: 
        print caseno 
f.close()

1030401
10302
10302
103003
1030415


## 如何從url 取得這兩個值?

In [93]:
link = "http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328" 

## 使用split

In [95]:
rear_substring = link.split("pkAtmMain=")[1] 
param = rear_substring.split("&tenderCaseNo=") 
pkAtmMain= param[0] 
caseno= param[1] 
print pkAtmMain,caseno 

51239291 0607L1030328


## re.match 範例

In [97]:
import re 
m = re.match(r"(?P<FirstName>\w+) (?P<LastName>\w+)", "David Chiu") 
print m.group("FirstName"), m.group("LastName") 

David Chiu


## 使用re.match

In [99]:
import re 
link = "http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328" 
m = re.match(r"([^ ]+)pkAtmMain=(?P<pkAtmMain>.+)&tenderCaseNo=(?P<tenderCaseNo>.+)", link) 
print m.group('pkAtmMain'), m.group('tenderCaseNo')

51239291 0607L1030328


## 讀取bid_list.txt 依序抓出標案內容

In [100]:
# -*- coding: utf-8 -*- 
from bs4 import BeautifulSoup 
import re, requests 

bid_list = open('bid_list.txt', 'r') 
for line in bid_list.readlines(): 
    pagelink = line.strip() 
    m = re.match(r"([^ ]+)pkAtmMain=(?P<pkAtmMain>.+)&tenderCaseNo=(?P<tenderCaseNo>.+)", pagelink) 
    filename = "%s_%s"%(m.group('pkAtmMain'), m.group('tenderCaseNo')) 
    request_get = requests.get(pagelink) 
    response = request_get.text.encode('utf8') 
    soup = BeautifulSoup(response) 
    printarea = soup.select('#printArea')[0] 
    bid_detail = open("gov/%s.txt"%(filename), 'w') 
    bid_detail.write(printarea .prettify("utf-8")) 
    bid_detail.close() 
bid_list.close()

IOError: [Errno 2] No such file or directory: 'gov/51239371_4197.txt'